# Encoder-Decoder melhorado (Cap 2)

**Descrição:**
Este notebook apresenta uma versão **melhorada e didática** do modelo encoder–decoder, incorporando conceitos fundamentais do processamento de texto usados em modelos de linguagem modernos. Além da tokenização básica, o modelo passa a lidar explicitamente com **palavras desconhecidas** e **delimitação de sequências**, utilizando tokens especiais como `<|unk|>` e `<|endoftext|>`, conforme discutido no Capítulo 2 do livro *Build a Large Language Model (From Scratch)*.

**Objetivo:**
Demonstrar, de forma clara e passo a passo, como evoluir um encoder–decoder simples para um pipeline mais robusto de preparação de texto, capaz de:

* Construir um vocabulário a partir de um corpus
* Representar palavras desconhecidas de forma controlada
* Marcar explicitamente o final de uma frase
* Converter texto em tokens numéricos (encode)
* Reconstruir texto a partir desses tokens (decode)

O objetivo é aproximar o leitor das práticas reais usadas em LLMs, mantendo a simplicidade conceitual.

**Funcionamento:**

![Encoder-decoder melhorado](../../imagens/03_encode_decode_melhorado.png)

O funcionamento do encoder–decoder melhorado segue as etapas abaixo:

1. **Construção do corpus:**
   Um conjunto de frases é tokenizado e usado para criar um vocabulário global. Durante essa etapa, o token especial `<|endoftext|>` é adicionado ao final de cada frase, e o token `<|unk|>` é garantido no vocabulário.

2. **Encode (codificação):**
   Um novo texto é tokenizado e convertido em uma sequência de IDs numéricos.

   * Tokens presentes no vocabulário são mapeados diretamente.
   * Tokens desconhecidos são substituídos por `<|unk|>`.
   * O token `<|endoftext|>` é adicionado para indicar o fim da sequência.

3. **Decode (decodificação):**
   A sequência de token IDs é convertida de volta para tokens textuais e reconstruída em texto legível, ignorando o marcador de fim de texto.

Esse fluxo representa uma versão simplificada, porém conceitualmente fiel, do pré-processamento de texto utilizado em modelos de linguagem baseados em transformers.

## Implementação de um encoder-decoder melhorado

In [1]:
import re
from collections.abc import Iterable
from dataclasses import dataclass, field


@dataclass
class EncoderDecoderWithSpecialTokens:
    """
    Encoder-Decoder didático com tokens especiais:
    - <|unk|>        : palavra desconhecida
    - <|endoftext|>  : fim de frase
    """

    padrao: str = r"([,.!?;:]|\s)"

    # Tokens especiais (seguindo o livro)
    UNK_TOKEN: str = "<|unk|>"
    EOS_TOKEN: str = "<|endoftext|>"

    vocab: dict[str, int] = field(default_factory=dict)
    inv_vocab: dict[int, str] = field(default_factory=dict)

    # ==========================
    # Tokenização interna
    # ==========================
    def _tokenizar(self, texto: str) -> list[str]:
        split_raw = re.split(self.padrao, texto)
        tokens = [p.strip() for p in split_raw if p.strip()]
        return tokens

    # ==========================
    # 1) Gerar corpus
    # ==========================
    def gerar_corpus(self, frases: Iterable[str]) -> None:
        print("=== PASSO 1: Construção do corpus ===")

        todas_tokens: list[str] = []

        for i, frase in enumerate(frases, start=1):
            print(f"\nFrase {i}: {frase}")
            tokens = self._tokenizar(frase)
            tokens.append(self.EOS_TOKEN)  # marca fim da frase
            print("Tokens + <|endoftext|>:", tokens)
            todas_tokens.extend(tokens)

        print("\n=== PASSO 2: Tokens coletados ===")
        print(todas_tokens)

        print("\n=== PASSO 3: Construção do vocabulário ===")

        tokens_unicos = sorted(set(todas_tokens))

        # Garantir tokens especiais
        if self.UNK_TOKEN not in tokens_unicos:
            tokens_unicos.append(self.UNK_TOKEN)

        tokens_unicos = sorted(tokens_unicos)

        self.vocab = {tok: i for i, tok in enumerate(tokens_unicos)}
        self.inv_vocab = {i: tok for tok, i in self.vocab.items()}

        print("Vocabulário final:")
        for tok, idx in self.vocab.items():
            print(f"{tok:15s} -> {idx}")

    # ==========================
    # 2) Encode
    # ==========================
    def encode(self, texto: str) -> list[int]:
        if not self.vocab:
            raise ValueError("Vocabulário não criado. Execute gerar_corpus().")

        print("\n=== ENCODE ===")
        print("Texto:", texto)

        tokens = self._tokenizar(texto)
        tokens.append(self.EOS_TOKEN)

        print("Tokens identificados:", tokens)

        token_ids = []
        for tok in tokens:
            if tok in self.vocab:
                token_ids.append(self.vocab[tok])
            else:
                print(f"Token desconhecido encontrado: '{tok}' -> <|unk|>")
                token_ids.append(self.vocab[self.UNK_TOKEN])

        print("Token IDs:", token_ids)
        return token_ids

    # ==========================
    # 3) Decode
    # ==========================
    def decode(self, token_ids: list[int]) -> str:
        if not self.inv_vocab:
            raise ValueError("Vocabulário não criado.")

        print("\n=== DECODE ===")
        print("Token IDs:", token_ids)

        tokens = [self.inv_vocab[i] for i in token_ids]
        print("Tokens:", tokens)

        # Remove <|endoftext|> da reconstrução
        tokens = [t for t in tokens if t != self.EOS_TOKEN]

        texto = " ".join(tokens)
        texto = re.sub(r"\s+([,.!?;:])", r"\1", texto)

        print("Texto decodificado:", texto)
        return texto

## Geração do corpus

In [2]:
corpus = [
    "O gato sobe no tapete.",
    "O cachorro sobe na mesa.",
    "A aranha desce a parede.",
    "O gato desce da mesa.",
]

ed = EncoderDecoderWithSpecialTokens()
ed.gerar_corpus(corpus)

=== PASSO 1: Construção do corpus ===

Frase 1: O gato sobe no tapete.
Tokens + <|endoftext|>: ['O', 'gato', 'sobe', 'no', 'tapete', '.', '<|endoftext|>']

Frase 2: O cachorro sobe na mesa.
Tokens + <|endoftext|>: ['O', 'cachorro', 'sobe', 'na', 'mesa', '.', '<|endoftext|>']

Frase 3: A aranha desce a parede.
Tokens + <|endoftext|>: ['A', 'aranha', 'desce', 'a', 'parede', '.', '<|endoftext|>']

Frase 4: O gato desce da mesa.
Tokens + <|endoftext|>: ['O', 'gato', 'desce', 'da', 'mesa', '.', '<|endoftext|>']

=== PASSO 2: Tokens coletados ===
['O', 'gato', 'sobe', 'no', 'tapete', '.', '<|endoftext|>', 'O', 'cachorro', 'sobe', 'na', 'mesa', '.', '<|endoftext|>', 'A', 'aranha', 'desce', 'a', 'parede', '.', '<|endoftext|>', 'O', 'gato', 'desce', 'da', 'mesa', '.', '<|endoftext|>']

=== PASSO 3: Construção do vocabulário ===
Vocabulário final:
.               -> 0
<|endoftext|>   -> 1
<|unk|>         -> 2
A               -> 3
O               -> 4
a               -> 5
aranha          -> 6
cac

## Exemplo de uso do encode (com uma palavra desconhecida)

In [3]:
texto_novo = "O rato sobe na mesa."
token_ids = ed.encode(texto_novo)


=== ENCODE ===
Texto: O rato sobe na mesa.
Tokens identificados: ['O', 'rato', 'sobe', 'na', 'mesa', '.', '<|endoftext|>']
Token desconhecido encontrado: 'rato' -> <|unk|>
Token IDs: [4, 2, 15, 12, 11, 0, 1]


## Exemplo do decode (com a utilização do <|unk|> e <|endoftext|>

In [4]:
texto_recuperado = ed.decode(token_ids)


=== DECODE ===
Token IDs: [4, 2, 15, 12, 11, 0, 1]
Tokens: ['O', '<|unk|>', 'sobe', 'na', 'mesa', '.', '<|endoftext|>']
Texto decodificado: O <|unk|> sobe na mesa.
